In [25]:
"""
Sample an image from a PixelCNN.
"""

import random

from PIL import Image
import numpy as np
import torch

from examples.mnist.model import Generator, make_vq_vae
from matplotlib import pyplot as plt

DEVICE = torch.device('cuda')

In [2]:
def sample_softmax(probs):
    number = random.random()
    for i, x in enumerate(probs):
        number -= x
        if number <= 0:
            return i
    return len(probs) - 1


In [27]:

vae = make_vq_vae()
vae.load_state_dict(torch.load('test_run_mnist/i=900_vae.pt', map_location='cuda'))
vae.to(DEVICE)
vae.eval()
generator = Generator()
generator.load_state_dict(torch.load('test_run_mnist/i=900_gen.pt', map_location='cuda'))
generator.to(DEVICE)

inputs = np.zeros([16, 7, 7], dtype=np.longlong)
for row in range(7):
    for col in range(7):
        with torch.no_grad():
            outputs = torch.softmax(generator(torch.from_numpy(inputs).to(DEVICE)), dim=1)
            for i, out in enumerate(outputs.cpu().numpy()):
                probs = out[:, row, col]
                inputs[i, row, col] = sample_softmax(probs)
    print('done row', row)
embedded = vae.encoders[0].vq.embed(torch.from_numpy(inputs).to(DEVICE))
decoded = torch.clamp(vae.decoders[0]([embedded]), 0, 1).detach().cpu().numpy()
decoded = np.concatenate(decoded, axis=1)
# decoded = decoded.reshape((4,4,32,32))
decoded=(decoded * 255).astype(np.uint8)[0]
# decoded=decoded.reshape((int(32*4),int(32*4)))
Image.fromarray(decoded).save('samples_2.png')


done row 0
done row 1
done row 2
done row 3
done row 4
done row 5
done row 6
